<a href="https://colab.research.google.com/github/LucasColas/ML01-Machine-Learning-for-everyone/blob/main/TD3_Naive_Bayes_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML01 : TD3 - Classifieur de Bayes empirique et Naïf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
import scipy.stats as stats

np.random.seed(407)

# 1. Contexte et introduction du Classifieur de Bayes Naïf

Plaçons nous dans un contexte général où nous disposons d'un ensemble de $n$ observations labélisées (une base d'apprentissage contenant $n$ observations dont nous connaissons les classes). Notons $\mathcal{S} = \left\{\left(Y_i,X_{i}\right), i = 1,\dots,n\right\}$ cette base d'apprentissage et $K\geq 2$ le nombre de classes possibles à diagnostiquer (à prédire). Pour chaque observation $i\in\{1,\dots,n\}$, 
- $X_{i} = [X_{i1},\dots,X_{id}]$ correspondra au profil caractérisant l'observation $i$ ($X_{i}$ est un vecteur aléatoire composé de $d$ variables aléatoires).
- $Y_{i}\in\{1,\dots,K\}$ correspondra à la variable aléatoire (catégorielle) caractérisant la classe de l'observation $i$.

Nous nous plaçons de plus dans le contexte général où le modèles génératif ayant permis de générer les observations de la base d'apprentissage est inconnu. En d'autres termes, nous ne connaissons pas les densités de distributions des variables aléatoire $(Y_i,X_i)$.

Notre objectif central reste le même que pour les précédents TDs : Nous souhaitons apprendre un classifieur (une règle de décision) à partir des données d'apprentissage permettant de prédire (diagnostiquer) la classe $Y_i$ d'une observation $i$ à partir du profil $X_i = [X_{i1},\dots,X_{id}]$ caractérisant cette observation.
Plus particulièrement dans ce TD, nous souhaitons apprendre le classifieur de Bayes Naïf.

**Le classifieur de Bayes Naïf** est un modèle probabiliste charchant à approximer la formule de Bayes en supposant que toutes les variables explicatives $\{X_{i1},\dots,X_{id}\}$, composant le profil $X_i$ de chaque observation, sont statistiquement indépendantes les unes des autres.


**Question 1.** Écrire la formule de Bayes permettant de modéliser la probabilité qu'une observation $i$ soit dans la classe $k\in\{1,\dots,K\}$ à partir du profile $X_i = [X_{i1},\dots,X_{id}]$. En déduire la règle de décision du classifieur de Bayes permettant d'attribuer la classe la plus probable à une observation $X_i = x_i$ et simplifier là. Cette règle de décision simplifiée est communément appelée la "MAP rule" (règle de décision du Maximum A Posteriori).

- Réponse à la Question 1 : 
\begin{equation}
        \begin{aligned}
            \mathbb{P}(Y_i=k|X_i=x_i)  &\;=\; \frac{\mathbb{P}(Y_i=k) \; \mathbb{P}(X_{i}=x_i|Y_i=k)}{\mathbb{P}(X_{i}=x_i)} \\
             &\;=\; \frac{\mathbb{P}(Y_i=k) \; \mathbb{P}(X_{i1}=x_{i1},\dots,X_{id}=x_{id}|Y_i=k)}{ \mathbb{P}(X_{i1}=x_{i1},\dots,X_{id}=x_{id}) } .
            \end{aligned}
        \end{equation}


        
    

\begin{equation}
        \delta^B : x_i \mapsto \mathrm{argmax}_{k\in\{1,\dots,K\}} \; \mathbb{P}(Y_i=k|X_i=x_i)
        \end{equation}

**Question 1.2.** En supposant que toutes les variables explicatives $\{X_{i1},\dots,X_{id}\}$ composant le profil $X_i$ de chaque observation sont indépendantes les unes des autres, comment pouvons-nous simplifier cette règle de décision du Maximum A Posteriori ?

- Réponse à la Question 2 : 

\begin{equation}
            \mathbb{P}(X_{i1},\dots,X_{id}|Y_i=k) \; = \; \prod_{j=1}^d \mathbb{P}(X_{ij}|Y_i=k).
    \end{equation}

\begin{equation}
        \begin{aligned}
            \mathbb{P}(Y_i=k|X_i=x_i)  &\;=\; \frac{\mathbb{P}(Y_i=k) \; \mathbb{P}(X_{i}=x_i|Y_i=k)}{\mathbb{P}(X_{i}=x_i)} \\
             &\;=\; \frac{\mathbb{P}(Y_i=k) \;  \prod_{j=1}^d \mathbb{P}(X_{ij} = x_{ij}|Y_i=k)}{ \prod_{j=1}^d \mathbb{P}(X_{ij} = x_{ij}) } .
            \end{aligned}
        \end{equation}
    La règle de décision du Maximum A Posteriori devient alors :
    \begin{equation}
        \delta^B : x_i \mapsto \mathrm{argmax}_{k\in\{1,\dots,K\}} \; \frac{\mathbb{P}(Y_i=k) \;  \prod_{j=1}^d \mathbb{P}(X_{ij} = x_{ij}|Y_i=k)}{ \prod_{j=1}^d \mathbb{P}(X_{ij} = x_{ij}) }
    \end{equation}


    

**Intérêt de faire l'hypothèse d'indépendance entre variables.** Pour rappel, nous ne connaissons pas les densités de distributions des variables aléatoires $(Y_i,X_i)$. En particulier, nous ne connaissons pas la loi jointe $\mathbb{P}(X_{i1},\dots,X_{id}|Y_i=k)$ des variables caractéristiques dans chaque classe $k$ (également appelée la vraisemblance de chaque classe). Lorsque nous disposons d'un grand nombre $d$ de variables explicatives, l'estimation de cette la loi jointe $\mathbb{P}(X_{i1},\dots,X_{id}|Y_i=k)$ est très difficile (surtout lorsque nous travaillons avec des variables explicatives numériques (comme la température d'un patient) et d'autres catégorielles (comme l'état fumeur/non fumeur d'un patient). 

D'après la question 2, l'hypothèse naïve d'indépendance entre variables descrptives permet de simplifier cette étape de modélisation de la vraisemblance puisque cela nous ramène à modéliser indépendamment la vraisemblance $\mathbb{P}(X_{ij}|Y_i=k)$ de chaque variable $j\in\{1,\dots,d\}$ dans chaque classe $k\in\{1,\dots,K\}$.

# 2. Apprentissage du classifieur de Bayes Naïf Empirique

D'après la question 2, nous avons besoin de modéliser indépendamment la vraisemblance $\mathbb{P}(X_{ij}|Y_i=k)$ de chaque variable descriptive $j\in\{1,\dots,d\}$ dans chaque classe $k\in\{1,\dots,K\}$, ainsi que les probabilités à priori $\mathbb{P}(Y_i=k)$ de chaque classe, le tout à partir des données d'apprentissage.

**Question 2.1.** Concernant l'estimation des probabilités à priori de chaque classe, une approche usuelle est de les approximer en calculant les proportions par classe dans la base d'apprentissage. Autrement dit, les probabilités à priori $\mathbb{P}(Y_i=k)$ de chaque classe $k\in\{1,\dots,K\}$ sont estimées par :
$$
\hat{\mathbb{P}}(Y_i=k) \; = \; \frac{1}{n} \sum_{i\in\mathcal{I}} \mathbf{1}_{\{Y_i = k\}}.
$$
Implémenter une fonction permetant d'estimer les probabilités à priori $\mathbb{P}(Y_i=k)$ de chaque classe $k\in\{1,\dots,K\}$ à partir des données d'apprentissage.

In [ ]:
# RÉPONSE À LA QUESTION 2.1 :

# Nous appelerons "priori_hat" le vecteur contenant l'estimation des probabilités à priori de chaque classe.

def estimation_priors(K,Y):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    Y : np.array
        Real labels.

    Returns
    -------
    priori_hat : Array of floats
        Class proportions.
    '''
    
    priori_hat = np.zeros((1,K))
    for i in range(K):
      priori_hat[0,i] = np.sum(Y == i+1)/Y.shape[0]

    
    return priori_hat
    


**Question 2.2.** Concernant la modélisation des vraisemblance $\mathbb{P}(X_{ij}|Y_i=k)$ de chaque variable descriptive $j\in\{1,\dots,d\}$ dans chaque classe, une approche usuelle est la suivante :

- Si la variable descriptive $j$ est une variable catégorielle à $T\geq 2$ valeurs possibles (par exemple est-ce qu'un patient est fumeur ou non), nous considérons que $(X_{ij}|Y_i = k) \sim \mathrm{Cat}(T,[p_{j,k,1},\dots,p_{j,k,T}])$, où quelque soit $t \in \{1,\dots,T\}$,   $p_{j,k,t} = \mathbb{P}(X_{ij} = t|Y_i=k)$.
  Comme nous l'avons rappelé dans la première partie, la loi de $(X_{ij}|Y_i = k)$ n'est pas connue est donc nous ne connaissons pas les paramètres $[p_{j,k,1},\dots,p_{j,k,T}]$. À partir de la base d'apprentissage, nous estimons donc ces paramètres de sorte que nos paramètres estimés rendent les observations de la base d'apprentissage les plus plausibles possible. Autrement dit nous les estimons par la méthode du maximum de vraisemblance. Ceci revient à calculer, pour chaque variable catégorielle $j$, pour chaque classe $k\in\{1,\dots,K\}$ et pour chaque catégorie $t \in \{1,\dots,T\}$ :
  

$$  
  \hat{p}_{j,k,t} \; = \; \hat{\mathbb{P}}(X_{ij} = t \mid Y_i=k) \; = \; \frac{1}{n_k} \sum_{i\in\mathcal{I_k}} \mathbf{1}_{\{x_{ij} = t\}},
$$


  
où $I_k$ correspond aux observations de la base d'apprentissage issues de la classe $k$ et $n_k$ correspond au nombre des ces observations issues de la classe $k$.
  

- Si la variable descriptive $j$ est une variable numérique, une approche usuelle est alors de modéliser les distributions de $(X_{ij}|Y_i = k)$ par des gaussiennes dont les moyennes et variances sont à estimer à partir de la base d'apprentissage, en cherchant à maximiser la vraisemblance par classe. Autrement dit, on estime que pour chaque variable numérique $j$ et pour chaque classe $k\in\{1,\dots,K\}$, $(X_{ij}|Y_i = k) \sim \mathcal{N}(\hat{\mu}_{jk},\hat{\sigma}_{jk}^2)$, où d'après la question 3 du TD2, les paramètres $\left\{\hat{\mu}_{jk},\hat{\sigma}_{jk}^2\right\}$ sont estimés par

$$
\hat{\mu}_{jk} = \frac{1}{n_k} \sum_{i\in I_k} x_{ij} 
\quad \quad \text{et} \quad \quad
\hat{\sigma}_{jk} = \frac{1}{n_k} \sum_{i\in I_k} \left(x_{ij} - \hat{\mu}_{jk} \right)^2.
$$
    
  De ce fait, la vraisemblance $\hat{\mathbb{P}}(X_{ij} = x_{ij} \mid Y_i=k)$ de chaque variable descriptive numérique $j$ dans chaque classe $k$ est estimée par
  
$$
    \hat{\mathbb{P}}(X_{ij} = x_{ij} \mid Y_i=k) \;=\; \frac{1}{\hat{\sigma}_{jk} \sqrt{2\pi}} \, \mathrm{exp}\left( -\frac{(x_{ij}-\hat{\mu}_{jk})^2}{2\hat{\sigma}_{jk}^2}\right).
$$
    
    
- Question : Implémenter une fonction permettant d'estimer les paramètres des vraisemblances d'une variable catégorielle $j$ dans chaque classe $k$. Implémenter ensuite une fonction permettant d'estimer les paramètres des vraisemblances d'une variable numérique $j$ dans chaque classe $k$.

In [3]:
# RÉPONSE À LA QUESTION 2.2 :

# Pour une variable catégorielle :

def estimation_vraisemblance_cat(Xj,Y,K,T):
    '''
    Parameters
    ----------
    Xj : np.array
        Vecteur contenant les données d'apprentissage de la variable j.
    Y : np.array
        Labels des observations de la base d'apprentissage
    K : int
        Number of classes.
    T : int
        Number of discrete profiles for the categorical variable Xj.

    Returns
    -------
    pHatj : Array of floats
        Estimation des vraisemblance catégorielles dans chaque classe.
    '''
    
    pHatj = np.zeros((K,T))
    
    # .... À COMPLETER. ...
    for k in range(K):
      nk = np.sum(Y==k+1)
      Ik = np.where(Y == k+1)
      for t in range(T):
            pHatj[k,t] = np.sum(Xj[Ik[0]]==t)/nk
    
    return pHatj
    
    
    
    
    

# Pour une variable numérique :

def estimation_vraisemblance_num(Xj,Y,K):
    '''
    Parameters
    ----------
    Xj : np.array
        Vecteur contenant les données d'apprentissage de la variable j.
    Y : np.array
        Labels des observations de la base d'apprentissage
    K : int
        Number of classes.

    Returns
    -------
    mu_j : Array of floats
        Estimation des moyennes des gaussiennes dans chaque classe.
    sigma_j : Array of floats
        Estimation des écart-types des gaussiennes dans chaque classe.
    '''

    mu_j = np.zeros((1,K))
    sigma_j = np.zeros((1,K))
    
    # .... À COMPLETER. ...
    for k in range(K):
      Ik = np.where(Y==(k+1))
      mu_j[0,k] = np.mean(Xj[Ik[0]])
      sigma_j[0,k] = np.std(Xj[Ik[0]])

    
    return mu_j, sigma_j

# 3. Application du classifieur de Bayes Naïf Empirique

Compiler la cellule ci-dessous permettant de générer une base de données d'apprentissage contenant nTrain = 100 observations et une base de test contenant 900 observations. 
- Chaque observation est décrite par 3 variables : les deux premières variables sont numériques et la 3ème variable est une variable catégorielle avec $T=4$ catégories possibles.
- Nous avons $K=2$ classes à prédire à partir de ces 3 variables descriptives.

In [4]:
# Génération base de données

# RÉPONSE À LA QUESTION 1:

np.random.seed(407)

K = 2

mu11 = 37.8
mu12 = 39
sigma11 = 0.4
sigma12 = 0.3

mu21 = 7
mu22 = 8
sigma21 = 0.7
sigma22 = 1

T = 4
pHat = np.zeros((K,T))
pHat[0,0]=0.1 
pHat[0,1]=0.2 
pHat[0,2]=0.5 
pHat[0,3]=1-(pHat[0,0]+pHat[0,1]+pHat[0,2]) 
pHat[1,0]=0.3 
pHat[1,1]=0.1 
pHat[1,2]=0.2 
pHat[1,3]=1-(pHat[1,0]+pHat[1,1]+pHat[1,2])     

priori_2 = 0.3


# Génération de la base d'apprentissage :

nTrain = 100
YTrain = np.zeros((nTrain,1))
XTrain = np.zeros((nTrain,3))

for i in range(0,nTrain):
    YTrain[i,0] = int(np.random.binomial(1, priori_2, 1) + 1)
    if YTrain[i,0]==1:
        XTrain[i,0] = np.random.normal(mu11, sigma11, 1)
        XTrain[i,1] = np.random.normal(mu21, sigma21, 1)
        XTrain[i,2] = np.random.choice(T, 1, p=pHat[0]) 
    if YTrain[i,0]==2:
        XTrain[i,0] = np.random.normal(mu12, sigma12, 1)
        XTrain[i,1] = np.random.normal(mu22, sigma22, 1)
        XTrain[i,2] = np.random.choice(T, 1, p=pHat[1])
    

# Génération de la base de test :
    
nTest= 900
YTest = np.zeros((nTest,1))
XTest = np.zeros((nTest,3))

for i in range(0,nTest):
    YTest[i,0] = int(np.random.binomial(1, priori_2, 1) + 1)
    if YTest[i,0]==1:
        XTest[i,0] = np.random.normal(mu11, sigma11, 1)
        XTest[i,1] = np.random.normal(mu21, sigma21, 1)
        XTest[i,2] = np.random.choice(T, 1, p=pHat[0]) 
    if YTest[i,0]==2:
        XTest[i,0] = np.random.normal(mu12, sigma12, 1)
        XTest[i,1] = np.random.normal(mu22, sigma22, 1)
        XTest[i,2] = np.random.choice(T, 1, p=pHat[1])
        



**Question 3.1.** Estimer sur la base d'apprentissage les paramètres introduits dans la section 2, pour chaque variable indépendamment, à partir des trois fonctions implémentées lors des questions 2.1 et 2.2.

In [5]:
# RÉPONSE À LA QUESTION 3.1 :

# .... À COMPLETER. ...
# Estimation des probabilités à priori :
priori_hat = estimation_priors(K,YTrain)

# Estimation des paramètres des vraisemblances de chaque classe pour la variable 1 qui est numérique :
XTrain_1 = XTrain[:,0]
mu_1, sigma_1 = estimation_vraisemblance_num(XTrain_1,YTrain,K)

# Estimation des paramètres des vraisemblances de chaque classe pour la variable 2 qui est numérique :
XTrain_2 = XTrain[:,1]
mu_2, sigma_2 = estimation_vraisemblance_num(XTrain_2,YTrain,K)

# Estimation des paramètres des vraisemblances de chaque classe pour la variable 3 qui est catégorielle :
XTrain_3 = XTrain[:,2]
pHat3 = estimation_vraisemblance_cat(XTrain_3,YTrain,K,T)

**Question 3.2.** Appliquer le classifieur de Bayes Naïf sur les observations de la base de test et calculer le taux d'erreur global de ce classifieur.

In [6]:
# RÉPONSE À LA QUESTION 3.2 :

YhatTest = np.zeros((nTest,1))

for i in range(nTest):
    
    # Estimation des probabilités à posteriori dans chaque classe :
    Posteriori = np.zeros((1,K))
    for k in range(K):
        
        pX1k_xi = 1/(sigma_1[0,k]*np.sqrt(2*np.pi)) * np.exp(-(XTest[i,0] - mu_1[0,k])**2/(2*sigma_1[0,k]**2))
        pX2k_xi = 1/(sigma_2[0,k]*np.sqrt(2*np.pi)) * np.exp(-(XTest[i,1] - mu_2[0,k])**2/(2*sigma_2[0,k]**2))
        pX3k_xi = pHat3[k,int(XTest[i,2])]
        
        Posteriori[0,k] = priori_hat[0,k] * pX1k_xi * pX2k_xi * pX3k_xi
        
    # Règle de décision Maximum A Posteriori (MAP rule) pour prédire la classe :
    YhatTest[i] = np.argmax(Posteriori)+1
    

# Calcul du taux d'erreur global :
errGlobalTest = np.sum(YhatTest!=YTest)/nTest
print('Taux erreur global du classifieur de Bayes Naif sur la base de test =', errGlobalTest)


Taux erreur global du classifieur de Bayes Naif sur la base de test = 0.03888888888888889
